In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [691 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Ge

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("demographicsFilter").getOrCreate()

In [3]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/dataviz-curriculum/day_1/demographics.csv"
spark.sparkContext.addFile(url)
df = spark.read.option('header', 'true').csv(SparkFiles.get("demographics.csv"), inferSchema=True, sep=',')

# Show DataFrame
df.show()

+---+--------------------+---+------------+---------+--------+------------------+---------------+------+-------------+
| id|                name|age|height_meter|weight_kg|children|        occupation|academic_degree|salary|     location|
+---+--------------------+---+------------+---------+--------+------------------+---------------+------+-------------+
|  0|       Darlena Avila| 58|        1.87|       53|       1|     Choreographer|            PhD|    68| South Dakota|
|  1|            Yan Boyd| 65|         1.8|       40|       0|         Cellarman|       Bachelor|    73|     Delaware|
|  2|         Joette Lane| 32|         1.8|       73|       1|Veterinary Surgeon|         Master|    69| South Dakota|
|  3|        Jazmine Hunt| 61|        1.79|       89|       0|            Hawker|            PhD|    88|    Louisiana|
|  4|      Remedios Gomez| 23|        1.64|       51|       2|     Choreographer|       Bachelor|    83|West Virginia|
|  5|        Myung Brewer| 20|        1.68|     

In [4]:
# What occupation had the highest salary?
df.orderBy(df['salary'].desc()).show(1)

+---+--------------+---+------------+---------+--------+-----------------+---------------+------+--------+
| id|          name|age|height_meter|weight_kg|children|       occupation|academic_degree|salary|location|
+---+--------------+---+------------+---------+--------+-----------------+---------------+------+--------+
| 11|Shawnee Harmon| 66|        1.63|       78|       5|Medical Physicist|            PhD|    90|Delaware|
+---+--------------+---+------------+---------+--------+-----------------+---------------+------+--------+
only showing top 1 row



In [5]:
# What occupation had the lowest salary?
df.orderBy(df['salary'].asc()).show(1)

+---+---------------+---+------------+---------+--------+-----------------+---------------+------+---------+
| id|           name|age|height_meter|weight_kg|children|       occupation|academic_degree|salary| location|
+---+---------------+---+------------+---------+--------+-----------------+---------------+------+---------+
|  7|Melodi Mcdowell| 56|         1.6|       42|       0|Lighthouse Keeper|         Master|    65|Louisiana|
+---+---------------+---+------------+---------+--------+-----------------+---------------+------+---------+
only showing top 1 row



In [6]:
# What is the mean salary of this dataset?
from pyspark.sql.functions import avg
df.select(avg("salary")).show()

+-----------+
|avg(salary)|
+-----------+
|     77.738|
+-----------+



In [9]:
# What is the max and min of the Salary column?
from pyspark.sql.functions import max,min
df.select(max("salary")).show()
df.select(min("salary")).show()

+-----------+
|max(salary)|
+-----------+
|         90|
+-----------+

+-----------+
|min(salary)|
+-----------+
|         65|
+-----------+



In [10]:
# Show all of the occupations where salaries were above 80k
df.filter(df['salary']>80).show()

+---+--------------------+---+------------+---------+--------+--------------------+---------------+------+-------------+
| id|                name|age|height_meter|weight_kg|children|          occupation|academic_degree|salary|     location|
+---+--------------------+---+------------+---------+--------+--------------------+---------------+------+-------------+
|  3|        Jazmine Hunt| 61|        1.79|       89|       0|              Hawker|            PhD|    88|    Louisiana|
|  4|      Remedios Gomez| 23|        1.64|       51|       2|       Choreographer|       Bachelor|    83|West Virginia|
|  8|Charlesetta Steve...| 30|        1.62|       44|       3|          Millwright|         Master|    87|    Louisiana|
| 11|      Shawnee Harmon| 66|        1.63|       78|       5|   Medical Physicist|            PhD|    90|     Delaware|
| 17|   Bernardina Strong| 34|        1.55|       78|       1|           Scientist|            PhD|    90| South Dakota|
| 21|      Kemberly Yates| 41|  

In [21]:
# BONUS
# What is the average age and height for each academic degree type?
# HINT: You will need to use `groupby` to solve this
df.groupby('academic_degree').avg('age').show()
df.groupby('academic_degree').avg('height_meter').show()

+---------------+------------------+
|academic_degree|          avg(age)|
+---------------+------------------+
|            PhD| 43.15976331360947|
|         Master|43.139318885448915|
|       Bachelor| 42.51032448377581|
+---------------+------------------+

+---------------+------------------+
|academic_degree| avg(height_meter)|
+---------------+------------------+
|            PhD|1.7438165680473379|
|         Master|1.7549226006191951|
|       Bachelor| 1.757227138643069|
+---------------+------------------+

